# Giris
Yeni çalışmadan herkese merhaba! Bu bölümde, ileri seviye konulardan biri olan Özelleştirme (Customization) bölümüne giriş yapıyoruz.

**Özelleştirme Temelleri: Tensörler ve İşlemler**<br>
Bu bölümde aşağıdaki konular üzerinde örnekler yapacağız:

- Gerekli paketlerin içe aktarımı
- Tensörleri oluşturma ve kullanma
- GPU hızlandırma kullanımı
- **tf.data.Dataset** ile bir veri hattı oluşturma

# TensorFlow'u içe aktar

In [1]:
import tensorflow as tf

# Tensors
**Tensör Nedir? (Tensors in TensorFlow)**
Tensör (Tensor), çok boyutlu dizileri (arrays) temsil eden temel veri yapısıdır ve TensorFlow gibi derin öğrenme kütüphanelerinde kullanılır. Tensörler, skaler, vektör, matris ve daha yüksek boyutlu dizilerin genelleştirilmiş halidir.

**Tensörlerin Temel Türleri**<br>
Tensörler, boyutlarına (rank) göre sınıflandırılır:

- Skaler (Rank-0 Tensör) → Tek bir sayıyı temsil eder.<br>
  Örnek: tf.constant(5)

- Vektör (Rank-1 Tensör) → Tek boyutlu dizidir.<br>
  Örnek: tf.constant([1, 2, 3])

- Matris (Rank-2 Tensör) → 2 boyutlu dizidir.<br>
  Örnek: tf.constant([[1, 2], [3, 4]])
  
- Yüksek Boyutlu Tensörler (Rank-3, 4, 5...) → Derin öğrenme ve görüntü işleme gibi alanlarda kullanılır.<br>
  Örnek: tf.constant([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

**Tensörlerin Özellikleri**
- Boyut (Shape) → Tensörün boyutlarını gösterir.<br>
  Örnek: (3, 2, 4) → 3 boyutlu tensör.
- Tip (Data Type) → TensorFlow'daki veri türlerini belirtir.<br>
  Örnek: tf.float32, tf.int32, tf.bool
- Cihaz Kullanımı → Tensörler CPU ve GPU üzerinde çalıştırılabilir.

In [2]:
print(tf.math.add(1,2))
print(tf.math.add([1,2], [3,4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1,2,3]))

print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [3]:
# Her tf.Tensor'un bir şekli ve bir veri türü vardır:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


NumPy dizileri ile tf.Tensors arasındaki en belirgin farklar şunlardır:
- Tensörler hızlandırıcı belleklerle (GPU, TPU gibi) desteklenebilir.
- Tensörler değişmezdir

## NumPy uyumluluğu
TensorFlow tf.Tensor ile NumPy ndarray arasında dönüşüm yapmak kolaydır:
- TensorFlow işlemleri NumPy ndarray'lerini otomatik olarak Tensörlere dönüştürür.
- NumPy işlemleri Tensörleri otomatik olarak NumPy ndarrays'lerine dönüştürür.

Tensörler, .numpy() metodu kullanılarak açıkça NumPy ndarrays'lerine dönüştürülür.

In [6]:
import numpy as np

ndarray = np.ones([3, 3])

print('TensorFlow işlemleri numpy dizilerini otomatik olarak Tensörlere dönüştürür')
tensor = tf.math.multiply(ndarray, 42)
print(tensor)

print('Ve NumPy işlemleri Tensörleri NumPy dizilerine otomatik olarak dönüştürür')
print(np.add(tensor, 1))

print('.numpy() yöntemi bir Tensörü açıkça bir numpy dizisine dönüştürür')
print(tensor.numpy())

TensorFlow işlemleri numpy dizilerini otomatik olarak Tensörlere dönüştürür
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
Ve NumPy işlemleri Tensörleri NumPy dizilerine otomatik olarak dönüştürür
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
.numpy() yöntemi bir Tensörü açıkça bir numpy dizisine dönüştürür
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


# GPU hızlandırma
Birçok TensorFlow işlemi hesaplama için GPU kullanılarak hızlandırılır. TensorFlow, herhangi bir açıklama olmaksızın, bir işlem için GPU'yu mu yoksa CPU'yu mu kullanacağına otomatik olarak karar verir; gerekirse tensörü CPU ve GPU belleği arasında kopyalar. Bir işlem tarafından üretilen tensörler genellikle işlemin yürütüldüğü cihazın belleği tarafından desteklenir. Örneğin:

In [7]:
x = tf.random.uniform([3, 3])

print('Kullanılabilir bir GPU var mı: ', tf.config.list_physical_devices('GPU')) # Ben suanda colab uzerinde ekran kartlarini kullanmadigim icin her hangi bir ekran kartina erisimim yok!
print("GPU #0'daki Tensör:", x.device.endswith('GPU:0'))

Kullanılabilir bir GPU var mı:  []
GPU #0'daki Tensör: False


## Cihaz adları
Tensor.device özelliği, tensörün içeriğini barındıran cihazın tam nitelikli dize adını sağlar. Bu ad, bu programın yürütüldüğü ana bilgisayarın ağ adresinin tanımlayıcısı ve o ana bilgisayardaki cihaz gibi birçok ayrıntıyı kodlar. Bu, TensorFlow programının dağıtılmış yürütülmesi için gereklidir. Tensör ana bilgisayardaki N'inci GPU'ya yerleştirilirse dize GPU:<N> ile biter.

## Açık cihaz yerleşimi

TensorFlow'da yerleştirme, bireysel işlemlerin yürütülmek üzere bir cihaza nasıl atandığını (yerleştirildiğini) ifade eder. Daha önce de belirtildiği gibi, açık bir kılavuz sağlanmadığında, TensorFlow bir işlemi hangi cihazda yürüteceğine otomatik olarak karar verir ve gerekirse tensörleri o cihaza kopyalar.

In [8]:
import time
def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.linalg.matmul(x, x)

  result = time.time()-start

  print('10 dongu: {:0.2f}ms'.format(1000*result))

# CPU uzerinde uygulamak
print('CPU uzerinde: ')
with tf.device('CPU:0'):
  X = tf.random.uniform([1000, 1000])
  assert x.device.endswith('CPU:0')
  time_matmul(x)

# GPU uzerinde uygulamak
if tf.config.list_physical_devices('GPU'):
  print('GPU uzerinde: ')
  with tf.device('GPU:0'): # 0 burada 1. GPU uzerinde calisacak demektir eger birden fazla GPU'ya sahipseniz GPU:1 yazark 2. gpu uzerinde calistirabilirsiniz!
    x = tf.device.endswith('GPU:0')
    time_matmul(x)

CPU uzerinde: 
10 dongu: 61.72ms


# Veri setleri
Bu bölüm, modelinize veri beslemek için bir işlem hattı oluşturmak amacıyla tf.data.Dataset API'sini kullanır.tf.data.Dataset, modelinizin eğitim veya değerlendirme döngülerini besleyecek basit, yeniden kullanılabilir parçalardan performanslı, karmaşık giriş hatları oluşturmak için kullanılır.

## Bir kaynak Veri Kümesi Oluşturun
**tf.data.Dataset.from_tensors**, **tf.data.Dataset.from_tensor_slices** gibi fabrika fonksiyonlarından birini kullanarak veya **tf.data.TextLineDataset** veya **tf.data.TFRecordDataset** gibi dosyalardan okuyan nesneleri kullanarak bir kaynak veri kümesi oluşturun.

In [9]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# CSV dosyasi olustur
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
  Line 2
  Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

## Dönüşümleri uygula
Veri kümesi kayıtlarına dönüşümler uygulamak için tf.data.Dataset.map, tf.data.Dataset.batch ve tf.data.Dataset.shuffle gibi dönüşüm fonksiyonlarını kullanın.

In [11]:
ds_tensors = ds_tensors.map(tf.math.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

## Tekrarla

In [12]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 9], shape=(2,), dtype=int32)
tf.Tensor([16  1], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'  Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'  Line 3' b'  '], shape=(2,), dtype=string)


# Sonuc
Bu bölümün sonuna geldik. Tensörler ile ilgili önemli bilgiler edindiğimiz bir çalışma oldu. Aynı zamanda, kodları CPU ve GPU üzerinde nasıl çalıştırabileceğimizi de görmüş olduk.

Umarım bu çalışma sizin için faydalı olmuştur. Eğer sormak istediğiniz herhangi bir soru varsa, aşağıdaki linklerden bana ulaşabilirsiniz.

İyi çalışmalar dilerim!

[Linkedin](https://www.linkedin.com/in/ihsancenkiz/)<br>
[Github](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)